#PharmacoForge

### **Before running this notebook, please update your runtime version to 2025.07 (Runtime > Change runtime type > Runtime version)**

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


In [ ]:
#@title Clone Github repo
import os

!git clone https://github.com/eflynn8/pharmacophore-diffusion

repo_url = 'https://github.com/eflynn8/pharmacophore-diffusion'
os.chdir(repo_url.split('/')[-1])

Cloning into 'pharmacophore-diffusion'...
remote: Enumerating objects: 936, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 936 (delta 65), reused 65 (delta 60), pack-reused 846 (from 1)
Receiving objects: 100% (936/936), 276.84 KiB | 1.43 MiB/s, done.
Resolving deltas: 100% (575/575), done.


In [ ]:
#@title Set up Colab environment

#!mamba install python=3.12 -y; pip install triton
!mamba install pytorch==2.4 torchdata=0.8.0 torchvision pytorch-cuda=12.1 -c pytorch -c nvidia
!mamba install -c dglteam/label/th24_cu121 dgl
!mamba install pytorch-cluster pytorch-scatter -c pyg -y
!mamba install -c conda-forge pytorch-lightning -y
!mamba install -c conda-forge rdkit -y
!mamba install -c conda-forge openbabel -y
!mamba install -c conda-forge pandas -y
!mamba install -c conda-forge biopython -y
!mamba install conda-forge::py3dmol

Streaming output truncated to the last 5000 lines.
Extracting  (78)  ⣾  [+] 3m:28.8s
Extracting  (78)  ⣾  [+] 3m:28.9s
Extracting  (78)  ⣾  [+] 3m:29.0s
Extracting  (78)  ⣾  [+] 3m:29.1s
Extracting  (78)  ⣾  [+] 3m:29.2s
Extracting  (78)  ⣾  [+] 3m:29.3s
Extracting  (78)  ⣾  [+] 3m:29.4s
Extracting  (78)  ⣾  [+] 3m:29.5s
Extracting  (78)  ⣾  [+] 3m:29.6s
Extracting  (78)  ⣾  [+] 3m:29.7s
Extracting  (78)  ⣾  [+] 3m:29.8s
Extracting  (78)  ⣾  [+] 3m:29.9s
Extracting  (78)  ⣾  [+] 3m:30.0s
Extracting  (78)  ⣾  [+] 3m:30.1s
Extracting  (78)  ⣾  [+] 3m:30.2s
Extracting  (78)  ⣾  [+] 3m:30.3s
Extracting  (78)  ⣾  [+] 3m:30.4s
Extracting  (78)  ⣾  [+] 3m:30.5s
Extracting  (78)  ⣾  [+] 3m:30.6s
Extracting  (78)  ⣾  [+] 3m:30.7s
Extracting  (78)  ⣾  [+] 3m:30.8s
Extracting  (78)  ⣾  [+] 3m:30.9s
Extracting  (78)  ⣾  [+] 3m:31.0s
Extracting  (78)  ⣾  [+] 3m:31.1s
Extracting  (78)  ⣾  [+] 3m:31.2s
Extracting  (78)  ⣾  [+] 3m:31.3s
Extracting  (78)  ⣾  [+] 3m:31.4s
Extracting  (78)  ⣾  [+] 3m:31.

In [ ]:
#@title Load pretrained model
model_dir = '/content/pharmacophore-diffusion/model_dir'
if not os.path.exists(model_dir):
  !wget -np -nH --cut-dirs=2 --reject 'index.html*' -P model_dir/checkpoints https://bits.csb.pitt.edu/files/pharmacoforge/last.ckpt
  !wget -np -nH --cut-dirs=2 --reject 'index.html*' -P model_dir https://bits.csb.pitt.edu/files/pharmacoforge/config.yaml

--2025-06-05 17:21:59--  https://bits.csb.pitt.edu/files/pharmacoforge/last.ckpt
Resolving bits.csb.pitt.edu (bits.csb.pitt.edu)... 136.142.4.139
Connecting to bits.csb.pitt.edu (bits.csb.pitt.edu)|136.142.4.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16800854 (16M)
Saving to: ‘model_dir/checkpoints/last.ckpt’

last.ckpt           100%[===================>]  16.02M  21.0MB/s    in 0.8s    

2025-06-05 17:22:01 (21.0 MB/s) - ‘model_dir/checkpoints/last.ckpt’ saved [16800854/16800854]

--2025-06-05 17:22:01--  https://bits.csb.pitt.edu/files/pharmacoforge/config.yaml
Resolving bits.csb.pitt.edu (bits.csb.pitt.edu)... 136.142.4.139
Connecting to bits.csb.pitt.edu (bits.csb.pitt.edu)|136.142.4.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1857 (1.8K)
Saving to: ‘model_dir/config.yaml’

config.yaml         100%[===================>]   1.81K  --.-KB/s    in 0s      

2025-06-05 17:22:01 (458 MB/s) - ‘model_dir/config.yaml’ sav

In [ ]:
#@title Upload the PDB file of the receptor

from google.colab import files

receptor_file = files.upload()
receptor_file = list(receptor_file.keys())[0]

Saving ace_receptor.pdb to ace_receptor.pdb


In [ ]:
#@title Select how to define the binding pocket

pocket_method = "Reference ligand file"  #@param ["Reference ligand file", "Residue list"]

In [ ]:
#@title Upload the SDF file of the ligand used to define the pocket OR provide a list of residues in the form chain ID:residue index (Example: "A:1" "A:2" "A:3")
use_ligand_ref = (pocket_method == "Reference ligand file")

if use_ligand_ref:
  ref_ligand_file = files.upload()
  ref_ligand_file = list(ref_ligand_file.keys())[0]

else:
  residue_list = ["A:1", "A:2", "A:3"]  # @param {type: "raw"}

Saving ace_crystal_ligand.sdf to ace_crystal_ligand.sdf


In [ ]:
#@title Specify parameters:
#@markdown - `samples_per_pocket`: Number of samples generated per pocket.
#@markdown - `pharm_sizes`: Number of pharmacophore centers in each sample. Length of list = samples_per_pocket

samples_per_pocket = 3  # @param {type: "integer"}
pharm_sizes = [3, 4, 5]  # @param {type: "raw"}


In [ ]:
#@title Generate pharmacophores
import subprocess

if use_ligand_ref:
  cmd = (
      f"python generate_pharmacophores.py {receptor_file} --ref_ligand_file {ref_ligand_file} "
      f"--model_dir {model_dir} "
      f"--samples_per_pocket {samples_per_pocket} "
      f"--pharm_sizes {' '.join(map(str, pharm_sizes))} "
      f"--use_ref_lig_com"
  )

else:
  residues = " ".join(f'"{r}"' for r in residue_list)
  cmd = (
      f"python generate_pharmacophores.py {receptor_file} --residue_list {residues} "
      f"--model_dir {model_dir} "
      f"--samples_per_pocket {samples_per_pocket} "
      f"--pharm_sizes {' '.join(map(str, pharm_sizes))} "
  )

result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("Command ran successfully")
    print(result.stdout)
else:
    print("Error running the command.")
    print(result.stderr)

radius = {
    'P': 1,  # Aromatic
    'S': 0.9,  # Hydrogen Donor
    'F': 1,  # Hydrogen Acceptor
    'C': 0.9,  # Hydrophobic
    'O': 0.8,  # Negative Ion
    'N': 0.8   # Positive Ion
}

def parse_xyz(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    all_pharmacophores = []
    while lines:
        num_feats = int(lines[0].strip())
        pharmacophore = [{'name': parts[0],
                          'coords': {'x': float(parts[1]), 'y': float(parts[2]), 'z': float(parts[3])},
                          'radius': radius[parts[0]]
                          } for parts in (line.split() for line in lines[1:num_feats+1])]
        all_pharmacophores.append(pharmacophore)
        lines = lines[num_feats+1:]

    return all_pharmacophores

pharmacophores = parse_xyz(f'generated_pharms/{receptor_file.split(".")[0]}/pharms.xyz')


Command ran successfully
device=device(type='cuda')
Pocket ace_receptor sampling time: 87.95 seconds
Pocket ace_receptor sampling time per pharmacophore: 29.32 seconds



In [ ]:
#@title Visualize Generated Pharmacophores
# @markdown CMap: Aromatic: Purple, HydrogenDonor: White, HydrogenAcceptor: Orange, Hydrophobic: Green, NegativeIon: Red, PositiveIon: Blue
import py3Dmol

cmap = {
    'P': 'purple',  # Aromatic
    'S': '0xf0f0f0',  # Hydrogen Donor
    'F': 'orange',  # Hydrogen Acceptor
    'C': 'green',  # Hydrophobic
    'O': 'red',  # Negative Ion
    'N': 'blue'   # Positive Ion
}

def add_pharmacophore_to_viewer(v, pharmacophore, viewer_index):
    for feat in pharmacophore:
        v.addSphere({
            'center': feat['coords'],
            'radius': feat['radius'],
            'color': cmap[feat['name']],
            'wireframe': True,
            'linewidth': 1.5
        }, viewer=(0, viewer_index))

v = py3Dmol.view(width="100%", height="400px", viewergrid=(1, len(pharmacophores)))
for i, pharmacophore in enumerate(pharmacophores):
    v.addLabel(i, {'position': {'x': 0, 'y': 0, 'z': 0}, 'useScreen': True, 'fontSize': 24}, viewer=(0, i))
    add_pharmacophore_to_viewer(v, pharmacophore, i)

v.zoomTo()
receptor_pdb = open(receptor_file, 'r').read()
v.addModel(receptor_pdb)
v.addSurface('VDW', {'opacity': 0.6, 'colorscheme': 'whiteCarbon'})
v.setStyle({'model': -1}, {'cartoon': {}, 'stick': {'radius': 0.15}})
v.show()



3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
#@title Load Pharmacophore into Pharmit
pharmacophore_number = 0  # @param {type: "integer"}
from google.colab import output
import json

name_mapping = {
    'P': 'Aromatic',
    'S': 'HydrogenDonor',
    'F': 'HydrogenAcceptor',
    'C': 'Hydrophobic',
    'O': 'NegativeIon',
    'N': 'PositiveIon'
}

json_list = []
for pharmacophore in pharmacophores:
    points = []
    for feat in pharmacophore:
        points.append({
            'enabled': True,
            'name': name_mapping[feat['name']],
            'radius': feat['radius'],
            'x': feat['coords']['x'],
            'y': feat['coords']['y'],
            'z': feat['coords']['z']
        })
    json_list.append(json.dumps({'points': points}, separators=(',', ':')))

javascript_code="""
//object for sending messages to a window, but only after we receive an ack
function Message(data, w, dest) {
    var curWindow = w;
    var curDest = dest;
    var curMsg = data;
    var isAcked = 0;

    function receiveMessage(event) {
        if (event.data == "ack2") {
            isAcked = 1;
        }
    }

    function check() {
        if (isAcked) {
            curWindow.postMessage(curMsg, curDest);
            curDest = "";
            curMsg = "";
            curWindow = null;
            isAcked = 0;
            window.removeEventListener("message", receiveMessage);
        } else if (curWindow) {
            curWindow.postMessage("ack", curDest);
            setTimeout(check, 250);
        }
    }

    window.addEventListener("message", receiveMessage);
    w.postMessage("ack", dest);
    setTimeout(check, 250);
}
var win = window.open("http://pharmit.csb.pitt.edu/search.html");
var msg = new Message(JSON.stringify("""+json_list[pharmacophore_number]+"""), win, '*');
"""
output.eval_js(javascript_code)